## Usando decoradores para mejorar el código

## Tabla de contenidos
***



***

## ¿Qué son los decoradores en Python?

Son un mecanismo para simplificar la manera en la que funciones y métodos son definidos cuando tienen que ser modificados luego de su definición original.

Cada vez que tenemos que aplicar una transformación a una función, tenemos que llamarla con la función que la modifica, y luego reasignarle el mismo nombre con el que la función originalmente fue definido.

Por ejemplo, si tenemos una función llamada `original` y luego tenemos otra función que cambia su comportamiento sobre esta, llamada `modifier`, tendríamos que escribir algo como lo siguiente:

In [4]:
def original():
    #Cuerpo de la función
    pass

def modifier(funcion):
    return funcion

original = modifier(original)

Note como cambiamos la función y luego le reasignamos el mismo nombre. El ejemplo anterior podría ser escrito como

In [5]:
@modifier
def original():
    #Cuerpo de la función
    pass

Esto significa que los decoradores son solo una forma de llamar lo que sea que está despues del decorador como un primer parámetro del decorador en si mismo, y el resultado va a ser lo que sea que retorne el decorador.

La syntaxis para los decoradores mejora la legibilidad de forma significativa, dado que ahora quien lee el código puede encontrar toda la definición de la función en un solo lugar.

En general, intente evitar reasignar valores a función que fue diseñada sin usar el syntax de los decoradores.

En cuanto a la terminología en Python, `modifier` es lo que se llama el **decorator** y `original` es la función decorada, llamada de vez en cuando **wraped** *object*

## Decoradores de funciones

Las funciones son probablemente la representación más simple de Python de un objeto que puede ser decorado. Podemos usar decoradores en funciones para aplicar todo tipo de lógicas en ellos - validar parámetros, chequear pre-condiciones, cambiar completamente su funcionamiento, modificar su huella, dejar en el caché resultados y más.

Como ejemplo, se creará un decorador que implementa un mecanismo `retry`, controlando un nivel particular de dominio de excepciones y reintentandolo un cierto número de veces.

In [16]:
from functools import wraps
import logging

class ControlledException(Exception):
    """A generic exception on the program's domain"""
    
    
def retry(operation):
    @wraps(operation)
    def wrapped(*args, **kwargs):
        last_raised = None
        RETRIES_LIMIT = 3
        for _ in range(RETRIES_LIMIT):
            try:
                return operation(*args, **kwargs)
            except ControlledException as e:
                logger.info("retrying %s", operation.__qualname__)
                last_raised = e
            
            raise last_raised
        
        return wrapped

El uso de _ en el `for loop` significa que el número es asignado a una variable que no nos interesa de momento, porque no es usada dentro del loop.

In [ ]:
@retry
def run_operation(task):
    """Run a particular task, simulating some failures on its execution"""
    
    return task.run()

## Decoradores para classes

Los decoradores de classes tienen consideraciones similares a los decoradores de funciones que ya se vieron. La única diferencia es que cuando se escribe código para este tipo de deceorador, hay que tener en consideración que se está recibiendo una clase como parámetro para el método *wrapped*, no otra función.

Algunos beneficios de decoradores que se utilizan en classes:
- Todos los beneficios de reusar código y el principio DRY. Un caso válido de un decorador podría ser el forzar que múltiples classes conformen una cierta interface o criterio.
- Podríamos crear classes más pequeñas o simples que luego pueden ser mejoradas por los decoradores.
- La lógica de transformación que necesitamos aplicar a una classe específica va a ser mucho más fácil de mantener si usamos un decorador, lo opuesto a lo más complicados acercamientos como meta-classes.

Recordando nuestro sistema de eventos para monitorear la plataforma, ahora necesitamos transformar los datos para cada evento y enviarlos a un sistema externo. Sin embargo, cada tipo de evento puede tener sus propias particularidades cuando se selecciona como enviar su información. En particular, el evento para un login puede contener información sensible como credenciales que queremos esconder. Otros campos como *timestamp* pueden requerir algunas transformaciones si es que queremos mostrar cierta información en un formato en específico. Un primer intento podría ser una classe que mapea cada evento particular y conoce como serializarlo.

In [11]:
from dataclasses import dataclass
from datetime import datetime

class LoginEventSerializer:
    def __init__(self, event):
        self.event = event
        
    def serialize(self):
        return {
            "username": self.event.username,
            "password": "**redacted**",
            "ip": self.event.ip,
            "timestamp" : self.event.timestamp.strftime("%Y-%m-%d%H:%M"),
        }
    
@dataclass
class LoginEvent:
    SERIALIZER = LoginEventSerializer
    
    username: str
    password: str
    ip: str
    timestamp: datetime
        
    def serialize(self):
        return self.SERIALIZER(self).serialize()

Si bien es una opción, existen algunos "peros":
- **Muchas classes**: A medida que aumenta el número de eventos, el número de serialización de classes va a crecer en la misma magnitud.
- **La solución no es lo suficientemente flexible**: Si queremos reusar partes de los componentes, tendríamos que extraerlo en una función, pero también llamar de forma repetida de múltiples classes.
- **Boilerplate**: El método `serialize()` va a tener que estar presente en todos los event classes, llamando el mismo código.

## Pasar argumentos a decoradores

Hay diversas maneras de implementar decoradores que toman argumentos. Una de ellas es crear decoradores como funciones anidadas con un nuevo nivel de *indirección*, haciendo caer todo en el decorador un nivel más profundo.

Otra opción es usar una clase para el decorador (esto es, implementar un objeto calleable que sigue actuando como decorador). En general, esta opción favorece la legibilidad, porque es más fácil pensar en términos de un objeto que en 3 o más funciones anidadas.

## Decoradores con funciones anidadas

## Buenos usos para los decoradores

- **Transformar parámetros**: Cambiar la signature de la función para exponer una API más bonita, mientras encapsulamos los detalles de como los parámetros son tratamos y transformados por debajo.
- **Rastreo de código:** Loggear la ejecución de una función con sus parámetros.
- **Validar parámetros:** Los decoradores pueden ser utilizados para validar los *type* de los parámetros. Con el uso de decoradores, podríamos hacer cumplir las precondiciones para nuestras abstracciones.
- **Implementar operaciones de retry**
- **Simplificar las clases moviendo lógica (repetitiva) a decoradores**

## Decoradores efectivos - evitando errores comunes

### Preservando data acerca del original wrapped object
Uno de los problemas más comunes cuando se aplica el decorador a una función es que algunas propiedades o atributos de la función original no son mantenidos, llevando a efectos indeseados, difíciles de trackear.

Usando el decorador `wraps`, podemos acceder la original, no modificada función bajo el atributo `__wrapped__`.

En general, para decoradores simples, la manera en la que utilizaríamos `functools.wraps` seguiría la siguiente fórmula general o estructura.

In [3]:
from functools import wraps

def decorator(original_function):
    @wraps(origina_function)
    def decorated_function(*args, **kwargs):
        #Modificaciones hechas por el decorador
        
        return original_function(*args, **kwargs)
    
    return decorated_function

Siempre use `functools.wraps` aplicada sobre la *wrapped function* cuando se cree un decorador, como se mostró recién.

## Lidiando con efectos secundarios en decoradores

Es recomendable evitar efectos secundarios en el cuerpo del decorador.

Todo lo que el decorador necesita hacer aparte de la función que está decorando debería ser colocado en el interior de la definición de la función o podrían haber problemas al momento de importar. Sin embargo, algunas veces estos efectos secundarios son requeridos para correr en *import time*.

### Manejo incorrecto de efectos secundarios en el decorador

## Requiriendo decoradores con efectos secundarios

## Creando decoradores que siempre van a funcionar

## Decoradores y clean code

## El principio DRY con decoradores

## Decoradores y la separación de concerns

## Análisis de buenos decoradores

- **Encapsulación o separacion de preocupaciones/concerns:** Un buen decorador debería separar efectivamente diferentes responsabilidades entre lo que hace y lo que está decorando.
- **Ortogonalidad:** Lo que el decorador hace debería ser independiente y estar tan desacoplado como sea posible del objeto que está decorando.
- **Reutilización:** Es deseable que el decorador pueda ser aplicado a múltiples tipos, y que no aparezca en una sola instancia de la función, porque eso significa que podría haber sido una simple función aparte. Tiene que ser lo suficientemente genérico.